# Tarea 2

Se procede a importar las librerías necesarias para la tarea, ya se explicará más adelante el uso de _combinations_.

In [1]:
import numpy as np
from os import listdir
from sklearn.neighbors import KNeighborsClassifier
from itertools import combinations

In [2]:
from pybalu.feature_extraction import lbp_features
from pybalu.feature_selection import clean, sfs
from pybalu.feature_transformation import normalize
from pybalu.io import imread

In [3]:
faces = [i for i in listdir('faces_ARLQ') if i.endswith('png')]
usable_faces = [i for i in faces if int(i[9:11]) < 8]
odd_faces_names = [i for i in usable_faces if int(i[5:8]) % 2]
even_faces_names = [i for i in usable_faces if not int(i[5:8]) % 2]

## Parte 1

Tras cargar todos los nombres de las imágenes en la carpeta _faces_ARLQ_ se procede a dividirlas entre pares e impares, y las labels se trasladan para que las pares no vayan de 1 a 100 con salto de 2, sino que de 0 a 49, correlativamente, lo anterior debido a una limitación de la implementación de SFS en pybalu que lo exigía.

In [4]:
training_names = [i for i in odd_faces_names if int(i[9:11]) > 1]
testing_names = [i for i in odd_faces_names if int(i[9:11]) <= 1]

In [5]:
training = [imread(f'faces_ARLQ/{i}') for i in training_names]
y_training = np.array([int(i[5:8]) // 2 for i in training_names])

In [6]:
testing = [imread(f'faces_ARLQ/{i}') for i in testing_names]
y_testing = np.array([int(i[5:8]) // 2 for i in testing_names])

Se probaron distintas combinaciones de _hdiv_ y _vdiv_, ambos entre los valores de 1 y 7. En un principio se probó con las más grandes (7x7) pero eran excesivamente lentas de procesar en el SFS, y se optó por probar las combinaciones extremas y más acotadas, esto es 5x1 y 1x5 iterando hasta 1x1. Si bien en varios experimentos se obtuvo un buen rendimiento (100% o muy cercano), el que mejor pondera tiempo de ejecución y rendimiento es el siguiente: 4x2

In [7]:
hdiv = 4
vdiv = 2
training_features = np.array([lbp_features(i, hdiv=hdiv, vdiv=vdiv) for i in training])
testing_features = np.array([lbp_features(i, hdiv=hdiv, vdiv=vdiv) for i in testing])

In [8]:
cleaned_features_indexes = clean(training_features)
cleaned_features = training_features[:, cleaned_features_indexes]
cleaned_testing_features = testing_features[:, cleaned_features_indexes] # mismo clean para testing
training_features.shape, cleaned_features.shape # se eliminó solo una

((300, 2048), (300, 2047))

In [9]:
normalized_features, norm_a, norm_b = normalize(cleaned_features)
normalized_testing_features = cleaned_testing_features * norm_a + norm_b # misma normalización para testing

In [10]:
N_FEATURES = 40
selected_features_indexes = sfs(normalized_features, y_training, n_features=N_FEATURES, method="fisher", show=True)

Selecting Features: 100%|██████████| 26.0/26.0 [03:31<00:00, 8.14s/ features]


Utilizar las features seleccionadas inmediatamente tras el SFS con 26 features entrega peores resultados que con menos features, de hecho el resultado que mostraré a continuación utiliza solo 24 features, sin embargo, esas 24 características no se obtienen de un SFS de N=24 por lo que empíricamente lo que se probó fue con combinaciones que dieran el mayor rendimiento.

Se computaron algunas combinaciones de las features seleccionadas por el SFS tales que maximizaran el resultado, considerando de ante mano que en muchas pruebas se alcanzaba el 100%. Para esto se utilizaron combinaciones de largo i-2 para cada i entre 0 y el tamaño del SFS realizado, por lo que el índice con el que se finaliza es el 26, para una combinación de largo 24.

In [11]:
for i in range(20, N_FEATURES + 1):
    s = []
    indexes = combinations(selected_features_indexes[:i], i - 2)
    for ixs in indexes:
        sf = normalized_features[:, ixs]
        n = KNeighborsClassifier(n_neighbors=1)
        n.fit(sf, y_training)
        ctf = testing_features[:, cleaned_features_indexes]
        ntf = ctf * norm_a + norm_b
        stf = ntf[:, ixs]
        yp = n.predict(stf)
        corr = (yp == y_testing).astype(int).sum()
        acc = corr / y_testing.size * 100
        s.append([acc, ixs])
    best_selected_indexes = max(s)[1]
    if max(s)[0] == 100:
        print(i)
        break

26


Como se observa se detiene anteriormente para evitar seguir si ya se consiguió un 100%, y esto se da con 24 características.

In [12]:
print(best_selected_indexes)

(376, 1851, 1726, 1595, 1655, 1144, 1275, 737, 1775, 1280, 391, 1670, 251, 1015, 632, 568, 511, 248, 286, 707, 195, 387, 763, 1795)


En caso de no querer correr el código anterior, considerando el tiempo de demora del SFS de 30 features, se puede descomentar la siguiente linea, que es el resultado de lo obtenido anteriormente.

In [13]:
# best_selected_indexes = (376, 1851, 1726, 1595, 1655, 1144, 1275, 737, 1775, 1280, 391, 1670, 251, 1015, 632, 568, 511, 248, 286, 707, 195, 387, 763, 1795)

Para probar el rendimiento (es lo mismo que se hace en cada iteración del _for_ interior del código anterior) se indexa la matriz de imagenes limpiada y normalizada con esos índices seleccionados, tanto para training como para testing, se entrena el clasificador de KNN y finalmente se evalúa el rendimiento

In [14]:
selected_features = normalized_features[:, best_selected_indexes]
selected_testing_features = normalized_testing_features[:, best_selected_indexes]

In [15]:
neighbors = KNeighborsClassifier(n_neighbors=1)
neighbors.fit(selected_features, y_training)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=1, p=2,
           weights='uniform')

In [16]:
y_prediction = neighbors.predict(selected_testing_features)

In [17]:
correct = (y_prediction == y_testing).astype(int).sum()
accuracy = correct / y_testing.size * 100
accuracy

100.0

## Parte 2

En esta parte, el procedimiento es muy similar a lo anterior, cargando las pares en vez de las impares, pero utilizando _best_selected_indexes_ que fue obtenido en el experimento anterior.

In [18]:
even_training_names = [i for i in even_faces_names if int(i[9:11]) > 1]
even_testing_names = [i for i in even_faces_names if int(i[9:11]) <= 1]

In [19]:
even_training = [imread(f'faces_ARLQ/{i}') for i in even_training_names]
even_y_training = np.array([int(i[5:8]) // 2 for i in even_training_names])
even_testing = [imread(f'faces_ARLQ/{i}') for i in even_testing_names]
even_y_testing = np.array([int(i[5:8]) // 2 for i in even_testing_names])

In [20]:
even_training_features = np.array([lbp_features(i, hdiv=hdiv, vdiv=vdiv) for i in even_training])
even_testing_features = np.array([lbp_features(i, hdiv=hdiv, vdiv=vdiv) for i in even_testing])

In [21]:
even_cleaned_training_features = even_training_features[:, cleaned_features_indexes]
even_cleaned_testing_features = even_testing_features[:, cleaned_features_indexes]

In [22]:
even_normalized_training_features, even_norm_a, even_norm_b = normalize(even_cleaned_training_features)
even_normalized_testing_features = even_cleaned_testing_features * even_norm_a + even_norm_b

In [23]:
even_selected_training_features = even_normalized_training_features[:, best_selected_indexes]
even_selected_testing_features = even_normalized_testing_features[:, best_selected_indexes]

In [24]:
knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(even_selected_training_features, even_y_training)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=1, p=2,
           weights='uniform')

In [25]:
even_y_prediction = knn.predict(even_selected_testing_features)

In [26]:
even_correct = (even_y_prediction == even_y_testing).astype(int).sum()
even_accuracy = even_correct / even_y_testing.size * 100
even_accuracy

100.0

Similarmente, como se observa, el rendimiento es perfecto con esos índices, y se obtiene un 100% en ambos experimentos, utilizando las features seleccionadas a partir de la parte impar del dataset.